The quality of extractions can often be improved by providing reference examples to the LLM.

:::{.callout-tip}
While this tutorial focuses how to use examples with a tool calling model, this technique is generally applicable, and will work
also with JSON more or prompt based techniques.
:::

In [4]:
from typing import Optional

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.pydantic_v1 import BaseModel, Field

# Определяем промпт: добавляем инструкций и доп. контекст
# Здесь мы можем:
# 1) Добавить примеров работы функций, для улучшения качества извлечения информации
# 2) Предоставить дополнительное информации из чего и что вы будете извлекать информацию
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Ты эксперт в извлечении информации из текста. "
            "Извлекай только релевантную информацию из текста. "
            "Если ты не знаешь значение аттрибута, "
            "который нужно извлечь, поставь аттрибуту null.",
        ),
        MessagesPlaceholder("examples"),  # <---- Примеры работы
        ("human", "{text}"),
    ]
)

Test out the template:

In [5]:
from langchain_core.messages import (
    HumanMessage,
)

prompt.invoke(
    {"text": "this is some text", "examples": [HumanMessage(content="testing 1 2 3")]}
)

ChatPromptValue(messages=[SystemMessage(content='Ты эксперт в извлечении информации из текста. Извлекай только релевантную информацию из текста. Если ты не знаешь значение аттрибута, который нужно извлечь, поставь аттрибуту null.'), HumanMessage(content='testing 1 2 3'), HumanMessage(content='this is some text')])

## Define the schema

Let's re-use the person schema from the quickstart.

In [6]:
from typing import List, Optional

from langchain_core.pydantic_v1 import BaseModel, Field


class Person(BaseModel):
    """Информация о человека."""

    # ^ Док-строка выше, подкладывается в описании функции
    # и может помочь в улучшении результатов от LLM

    # Заметьте:
    # 1. Каждое поле опциональное -- это позволяет LLM не извлекать поля, которые не описаны
    # 2. Каждое поле имеет поле description — это подкладывается в описание аргументов функции
    # и может помочь в улучшении результатов
    name: Optional[str] = Field(..., description="Имя человека")
    hair_color: Optional[str] = Field(
        ..., description="Цвет волос человека, заполни если известен"
    )
    height_in_meters: Optional[float] = Field(
        ..., description="Высота человека в метрах."
    )


class Data(BaseModel):
    """Информация о людях."""

    # Создаем модель, чтобы мы могли извлечь информацию о нескольких людях
    people: List[Person]

## Define reference examples

Examples can be defined as a list of input-output pairs. 

Each example contains an example `input` text and an example `output` showing what should be extracted from the text.

:::{.callout-important}
This is a bit in the weeds, so feel free to ignore if you don't get it!

The format of the example needs to match the API used (e.g., tool calling or JSON mode etc.).

Here, the formatted examples will match the format expected for the tool calling API since that's what we're using.
:::

In [7]:
import itertools
from typing import List, TypedDict

from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    FunctionMessage,
    HumanMessage,
)
from langchain_core.pydantic_v1 import BaseModel


class Example(TypedDict):
    """Пример работы функций."""

    input: str  # Пример вызова
    function_calls: List[BaseModel]  # Pydantic модель, с примером извлечения
    function_outputs: List[str]


def tool_example_to_messages(example: Example) -> List[BaseMessage]:
    """Превращаем примеры вызовов функций в историю сообщений"""
    messages: List[BaseMessage] = [HumanMessage(content=example["input"])]
    for function_call, function_output in itertools.zip_longest(
        example["function_calls"], example.get("function_outputs", [])
    ):
        messages.append(
            AIMessage(
                content="",
                additional_kwargs={
                    "function_call": {
                        # Названием функции в текущий момент соответствует The name of the function right now corresponds
                        # to the name of the pydantic model
                        # This is implicit in the API right now,
                        # and will be improved over time.
                        "name": function_call.__class__.__name__,
                        "arguments": function_call.dict(),
                    },
                },
            )
        )
        output = "You have correctly called this tool."
        if function_output:
            output = function_output
        messages.append(
            FunctionMessage(name=function_call.__class__.__name__, content=output)
        )
    return messages

Next let's define our examples and then convert them into message format.

In [8]:
examples = [
    (
        "Океан огромный и синий. Глубина более 20 000 футов. В нем много рыбы.",
        Data(people=[]),
    ),
    (
        "Фиона отправилась в путешествие из Испании во Францию",
        Data(people=[Person(name="Фиона", height_in_meters=None, hair_color=None)]),
    ),
]


messages = []

for text, function_call in examples:
    messages.extend(
        tool_example_to_messages({"input": text, "function_calls": [function_call]})
    )

Let's test out the prompt

In [9]:
prompt.invoke({"text": "this is some text", "examples": messages})

ChatPromptValue(messages=[SystemMessage(content='Ты эксперт в извлечении информации из текста. Извлекай только релевантную информацию из текста. Если ты не знаешь значение аттрибута, который нужно извлечь, поставь аттрибуту null.'), HumanMessage(content='Океан огромный и синий. Глубина более 20 000 футов. В нем много рыбы.'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'Data', 'arguments': {'people': []}}}), FunctionMessage(content='You have correctly called this tool.', name='Data'), HumanMessage(content='Фиона отправилась в путешествие из Испании во Францию'), AIMessage(content='', additional_kwargs={'function_call': {'name': 'Data', 'arguments': {'people': [{'name': 'Фиона', 'hair_color': None, 'height_in_meters': None}]}}}), FunctionMessage(content='You have correctly called this tool.', name='Data'), HumanMessage(content='this is some text')])

## Create an extractor
Here, we'll create an extractor using **gpt-4**.

In [12]:
from langchain_community.chat_models.gigachat import GigaChat

llm = GigaChat(
    verify_ssl_certs=False,
    timeout=6000,
    model="GigaChat-Pro",
    temperature=0.01,
)


runnable = prompt | llm.with_structured_output(
    schema=Data,
    include_raw=False,
)

## Without examples 😿

Notice that even though we're using gpt-4, it's failing with a **very simple** test case!

In [13]:
for _ in range(5):
    text = "Если бы у Земли были бы волосы, они были бы синими"
    print(runnable.invoke({"text": text, "examples": []}))

Giga generation stopped with reason: function_call


people=[Person(name='Земля', hair_color='синий', height_in_meters=None)]


Giga generation stopped with reason: function_call


people=[Person(name='Земля', hair_color='синий', height_in_meters=None)]


Giga generation stopped with reason: function_call


people=[Person(name='Земля', hair_color='синий', height_in_meters=None)]


Giga generation stopped with reason: function_call


people=[Person(name='Земля', hair_color='синий', height_in_meters=None)]


Giga generation stopped with reason: function_call


people=[Person(name='Земля', hair_color='синий', height_in_meters=None)]


## With examples 😻

Reference examples helps to fix the failure!

In [14]:
for _ in range(5):
    text = "Если бы у Земли были бы волосы, они были бы синими"
    print(runnable.invoke({"text": text, "examples": messages}))

Giga generation stopped with reason: function_call


people=[]


Giga generation stopped with reason: function_call


people=[]


Giga generation stopped with reason: function_call


people=[]


Giga generation stopped with reason: function_call


people=[]


Giga generation stopped with reason: function_call


people=[]


In [15]:
runnable.invoke(
    {
        "text": "Моё имя Джо, мои волосы синие",
        "examples": messages,
    }
)

Giga generation stopped with reason: function_call


Data(people=[Person(name='Джо', hair_color='синий', height_in_meters=None)])